Export 4 csvs `{country}_{metric}_{month yyyymmdd}.csv`

#### nrc
https://analytics.google.com/analytics/web/#/report/bf-roi-calculator/a63985451w105110532p181467072/_u.date00=20190301&_u.date01=20190331&_.bfType=9&_r.attrSel2=ddmProfile&_r.attrSel3=preset6&_r.tableMetrics=conversions&roicalculator_dataSet_ALL-dimensionExplorer-segmentExplorer.segmentId=bigfunnels.campaign_name&roicalculator_dataSet_ALL-table.plotKeys=%5B%5D&roicalculator_dataSet_ALL-table.secSegmentId=bigfunnels.source_medium&roicalculator_dataSet_ALL-table.rowStart=0&roicalculator_dataSet_ALL-table.rowCount=1000/

#### ndc
https://analytics.google.com/analytics/web/#/report/bf-roi-calculator/a63985451w105110532p181467072/_u.date00=20190301&_u.date01=20190331&_.bfType=5&_r.attrSel2=ddmProfile&_r.attrSel3=preset6&_r.tableMetrics=nocost&roicalculator_dataSet_ALL-dimensionExplorer-segmentExplorer.segmentId=bigfunnels.campaign_name&roicalculator_dataSet_ALL-table.plotKeys=%5B%5D&roicalculator_dataSet_ALL-table.secSegmentId=bigfunnels.source_medium&roicalculator_dataSet_ALL-table.rowStart=0&roicalculator_dataSet_ALL-table.rowCount=1000

#### dc
https://analytics.google.com/analytics/web/#/report/bf-roi-calculator/a63985451w105110532p181467072/_u.date00=20190301&_u.date01=20190331&_.bfType=6&_r.attrSel2=ddmProfile&_r.attrSel3=preset6&_r.tableMetrics=nocost&roicalculator_dataSet_ALL-dimensionExplorer-segmentExplorer.segmentId=bigfunnels.campaign_name&roicalculator_dataSet_ALL-table.plotKeys=%5B%5D&roicalculator_dataSet_ALL-table.secSegmentId=bigfunnels.source_medium&roicalculator_dataSet_ALL-table.rowStart=0&roicalculator_dataSet_ALL-table.rowCount=1000/

#### spend
https://analytics.google.com/analytics/web/#/report/bf-roi-calculator/a63985451w105110532p181467072/_u.date00=20190301&_u.date01=20190331&_.bfType=6&_r.attrSel2=ddmProfile&_r.attrSel3=preset6&_r.tableMetrics=conversions&roicalculator_dataSet_ALL-dimensionExplorer-segmentExplorer.segmentId=bigfunnels.campaign_name&roicalculator_dataSet_ALL-table.plotKeys=%5B%5D&roicalculator_dataSet_ALL-table.secSegmentId=bigfunnels.source_medium&roicalculator_dataSet_ALL-table.rowStart=0&roicalculator_dataSet_ALL-table.rowCount=1000/

In [9]:
import os
import datetime
from google.cloud import bigquery
import pandas as pd
import re
import itertools

private_key = 'serviceaccount.json'
# output_path = folder
bq_client = bigquery.Client()#.from_service_account_json(private_key)
storage_client = storage.Client()
project_id = 'projectidtest'

/Users/thesor/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/thesor/anaconda3/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/doc

In [78]:
def csv_to_bq(project_id, dataset_id, destination_table_id, bucket_name, source_file_name, table_schema):
    destination_blob_name = dataset_id + '/' + destination_table_id + '.csv'

    """
    Uploads a file to the bucket.
    https://cloud.google.com/storage/docs/uploading-objects
    """
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('Uploaded file {} to {}.'.format(source_file_name, destination_blob_name))

    """
    Load CSV from GCS to BQ
    https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv
    """
    dataset_ref = bq_client.dataset(dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.schema = table_schema
    job_config.skip_leading_rows = 1
    job_config.max_bad_records = 100
    job_config.field_delimiter = ","
    job_config.source_format = bigquery.SourceFormat.CSV # The source format defaults to CSV, so this line is optional.
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE # https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv#loading_csv_data_with_schema_auto-detection
    job_config.allow_quoted_newlines = True

    uri = 'gs://' + bucket_name + '/' + destination_blob_name

    load_job = bq_client.load_table_from_uri(
        uri,
        dataset_ref.table(destination_table_id),
        job_config=job_config)  # API request
    print('Starting job {} from {}'.format(load_job.job_id, uri))

    load_job.result()  # Waits for table load to complete.
    print('Job finished.')

    destination_table = bq_client.get_table(dataset_ref.table(destination_table_id))
    print('Resulted table has {} rows.'.format(destination_table.num_rows))
    
    return load_job

In [94]:
def workflow(country, month, metric):
    filename = country + '_' + metric + '_' + month + '.csv'
    
    try:
        df = pd.read_csv('./input/'+filename, skiprows=6, skipfooter=5, engine='python')
    except Exception as e:
        return print(str(e))

    if metric=='spend':
        original_columns = ["Spend"]
        value_columns = [metric]
    elif metric=='nrc':
        original_columns = ["Last Interaction Conversions", "Data-Driven Conversions", "Last Non-Direct Click Conversions",]
        value_columns = [
            "lc_" + metric,
            "dd_" + metric,
            "lndc_" + metric,
        ]
    else:
        original_columns = ["Last Interaction Conversions", "Last Interaction Conversion Value", "Data-Driven Conversions", "Data-Driven Conversion Value", "Last Non-Direct Click Conversions", "Last Non-Direct Click Conversion Value"]
        value_columns = [
            "lc_" + metric,
            "lc_" + metric + "_value",
            "dd_" + metric,
            "dd_" + metric + "_value",
            "lndc_" + metric,
            "lndc_" + metric + "_value"
        ]

    # new columns (dimensions)
    df['Month'] = month
    df['Country'] = country.upper()
    df['Source'], df['Medium'] = df['Source / Medium'].str.split(' / ',1).str
    
    # rename columns
    columns_to_rename = dict(zip(original_columns, value_columns))
    df.rename(columns=columns_to_rename, inplace=True)
    
    # format to number
    if metric=='spend':
        df[metric] = df[metric].replace('[\€,]', '', regex=True).astype(float)
    elif metric=='nrc':
        pass
    else:
        for col in value_columns[1::2]:
            df[col] = df[col].replace('[\€,]', '', regex=True).astype(float)

    # set the index
    df = df.set_index(['Country', 'Month', 'Source', 'Medium', 'Campaign'])
    
    # drop columns
    columns_to_drop = [col for col in df.columns.tolist() if col not in value_columns]
    df.drop(columns=columns_to_drop, inplace=True)
    
    # unpivot
    df = df.stack()
    df = df.to_frame('value')
    df.index.names = ['country', 'month', 'source', 'medium', 'campaign', 'metric']
    
    # doing the pivot 
#     for col in value_columns:
#         df[col] = df.loc[(slice(None), slice(None), slice(None), slice(None), slice(None), col), 'value']

    result_filename = 'model_comparison_'+filename
    df.to_csv('./output/'+result_filename, encoding='utf-8')#, header=['country', 'month', 'source', 'medium', 'campaign', 'metric', 'value'])
    
    print("Written CSV for", country, month, metric, './'+result_filename)
    
    project_id = 'bi-poc-189515'
    dataset_id = 'mrgreen'
    destination_table_id = 'model_comparison_' + country + '_' + metric + '_' + month
    bucket_name = 'bi_poc'
    source_file_name = './output/'+result_filename
    table_schema = [
        bigquery.SchemaField('country', 'STRING'),
        bigquery.SchemaField('month', 'STRING'),
        bigquery.SchemaField('source', 'STRING'),
        bigquery.SchemaField('medium', 'STRING'),
        bigquery.SchemaField('campaign', 'STRING'),
        bigquery.SchemaField('metric', 'STRING'),
        bigquery.SchemaField('value', 'FLOAT'),
    ]
    
    return csv_to_bq(project_id, dataset_id, destination_table_id, bucket_name, source_file_name, table_schema)
    

In [ ]:
# workflow('at','20190301','spend')

In [ ]:
countries=['at', 'de', 'dk', 'fi', 'ie', 'nl', 'no', 'se', 'uk']
months=['20190301']
metrics=['ndc','nrc','dc','spend']

for country, month, metric in itertools.product(countries, months, metrics):
    print(" ")
    print("Working on:",country, month, metric)
    workflow(country, month, metric)